<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Lembar Kerja MK ASD Kelas B TIF, Filkom UB</h1>

**************************************
Dosen Pengampu: Imam Cholissodin, S.Si., M.Kom. | imamcs@ub.ac.id | Fakultas Ilmu Komputer (Filkom), Universitas Brawijaya (UB) | Semester Genap 2021/2022 | Dibuat pada 1 Feb. 2020 | 22 Agustus 2021 | 8 Feb. 2022

Semoga Bermanfaat. Aamiin :D
**************************************


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ref:
# [0] https://towardsdatascience.com/develop-a-nlp-model-in-python-deploy-it-with-flask-step-by-step-744f3bdd7776
# [1] https://github.com/pemagrg1/NLP-Flask-Website
# [2] https://stackoverflow.com/questions/46454542/how-do-i-get-the-words-that-are-not-in-the-lexicon-of-a-cfg-grammar
# [3] etc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#FP - Run Web App pada Google Colab. untuk ParseTree & Typo App

## 1. Clone resource gdrive & dari github

In [ ]:
import os
from google.colab import output
os.chdir("/content/drive/My Drive")
path = "#Pengajaran Genap 2021-2022/ASD For Me/MyApp/MyFP"
if not os.path.exists(path):
  os.makedirs(path)
  os.chdir("/content/drive/My Drive/"+path)

  # download Web App dasar
  !gdown 1NFSfWCv_tLjSGwk6obznpvM1B6-UIml0
  !tar -xv -C './' -f './SmartAppRev5.tar'
  !mv ./SmartApp ./parsetree-search
  !rm ./SmartAppRev5.tar

  # download dok contoh yg didalamnya juga ada data kamus
  !gdown 1D7BfntXyW3uXb88USLqiuDs_IIJx2FxH
  !tar -xv -C './' -f './dok-contoh.tar'
  !cp -f ./dok-contoh/* ./.
  !rm ./dok-contoh.tar
  !rm -r ./dok-contoh

  os.makedirs("dok/pdfs")
  !mv ./dok_full.pdf ./dok/pdfs
  !mv ./dok_full2.pdf ./dok/pdfs
  os.makedirs("dok/txts")

  # import fileinput
  # with fileinput.FileInput('./SmartApp/templates/index.html', inplace=True, backup='.bak') as file:
  #   for line in file:
  #       print(line.replace('Smart Data Application', 'My Data Application'), end='')

output.clear()
os.chdir("/content/drive/My Drive/"+path)

!pwd
!ls -l -a --block-size=M

/content/drive/My Drive/#Pengajaran Genap 2021-2022/ASD For Me/MyApp/MyFP
total 10M
-rw------- 1 root root 1M May 16 13:55 db_kamus_kataYgDiTambahkanManual.bak.txt
-rw------- 1 root root 1M May 17 05:44 db_kamus_kataYgDiTambahkanManual.txt
drwx------ 2 root root 1M May 16 13:55 dok
-rw------- 1 root root 1M May 31 08:35 .env
-rw------- 1 root root 3M May 22 12:58 Indonesian_Manually_Tagged_Corpus2.tsv
-rw------- 1 root root 1M May 16 13:55 kamusEng.txt
-rw------- 1 root root 1M May 22 12:58 mygrammar.cfg
-rw------- 1 root root 1M May 16 13:55 myKBBI_tambahan.txt
-rw------- 1 root root 5M May 16 13:55 myKBBI.txt
-rw------- 1 root root 1M May 16 13:55 namaBulan.txt
-rw------- 1 root root 1M May 16 13:55 namaKota.txt
-rw------- 1 root root 1M May 16 13:55 namaNegara.txt
drwx------ 8 root root 1M May 31 16:10 parsetree-search
-rw------- 1 root root 1M May 17 07:37 spam.csv


In [ ]:
# Buatlah token dengan Sign-Up atau register ke htpps://dashboard.ngrok.com/auth
# get token dari keduanya, lalu masukkan pada kode di bawah ini, aktifkan kodenya lalu jalankan, lalu hide kembali token anda

# %%file ./.env
NGROK_AUTH_TOKEN = "29WLM5lBxIJRG0DeHUnWqDDMLzL_BUtxMMm9xFjZhfWXaKFz"

In [ ]:
# Instalasi Lib.
!pip install python-decouple
!pip install pyngrok --quiet
!pip install flask
!pip install flask_cors
!pip install db-sqlite3
!pip install freeport
!pip install cx_Oracle
!pip install flaskcode
!pip install autocorrect
output.clear()

# get token yang dihiden dalam file *.env
from decouple import config
NGROK_AUTH_TOKEN = config('NGROK_AUTH_TOKEN',default='')

## 2. Run "Web App" dgn tunnel

### Edit file app.py

In [ ]:
%%file ./parsetree-search/templates/fp_index2.html
{% extends "extends/base.html" %}
{% block title %}
    <title>Web Data App</title>
{% endblock title %}
{{ self.title() }}
    Home
{{ self.title() }}
<button onclick="window.location.href='/'" class="btn btn-outline btn-rounded btn-info">
    <i class="ti-arrow-left m-l-5"></i>
    <span>Back Home</span>
</button> Project 1

{{ self.title() }}
    Project 1

{% block content %}

	<div class="container">
      <div class="row">
        <div class="col-sm">
            <form action="{{ url_for('fppredict')}}" method="POST">
        		<p>Masukkan kalimat atau paragraf</p>
        		<!-- <input type="text" name="comment"/> -->
        		<textarea class="form-control" name="message" rows="4">{{ request.form['message'] }}</textarea>

        		<br/>

        		<input type="submit" class="btn-info" value="ParseTree_n_GetTypo">

	        </form>
        </div>
        <!-- <br/>
        <br/>
        <div class="col-sm">
            {% if prediction is defined %}
             <p style="color:blue;font-size:20;text-align: left;"><b>Hasil Prediksi (Spam/Not a Spam):</b></p>
        	 <div class="results">

            	{% if prediction == 1%}
            	<h2 style="color:red;">Spam</h2>
            	{% elif prediction == 0%}
            	<h2 style="color:blue;">Not a Spam (It is a Ham)</h2>
            	{% endif %}

        	</div>
        	{% endif %}
        </div> -->
        <br/>
        <br/>
        <div class="col-sm">
            {% if tree is defined %}

             <p style="color:blue;font-size:20;text-align: left;"><b>Hasil ParseTree:</b></p>
        	 <div class="results">
                {% autoescape false %}
            	<pre>{{ tree }}</pre>
                {% endautoescape %}

        	</div>

        	{% endif %}
        </div>
        <br/>
        <br/>
        <div class="col-sm">
            {% if typo is defined %}
             <p style="color:blue;font-size:20;text-align: left;"><b>Hasil GetTypo:</b></p>
        	 <div class="results">
            	<h2 style="color:red;">{{ typo }}</h2>
        	</div>
        	{% endif %}
        </div>
      </div>
    </div>

{% endblock content %}

Overwriting ./parsetree-search/templates/fp_index2.html


In [ ]:
!ls ./parsetree-search/templates/

404.html	  extends		   kode_cek_flask_reg.html
500.html	  fp_index2.html	   login.html
base_upload.html  fp_result.html	   MybigdataApps.html
bdc.html	  getsuhu_tipe_dua.html    MybigdataAppsNonPySpark.html
bigdataApps.html  getsuhu_tipe_empat.html  MybigdataAppsPySpark.html
biz2.html	  getsuhu_tipe_satu.html   register.html
biz.html	  getsuhu_tipe_tiga.html   twit.html
bli.html	  include		   twity.html
charts.html	  index.html		   upload.html
dashboard.html	  kode_cek_flask.html


In [ ]:
!wget https://raw.githubusercontent.com/susanli2016/SMS-Message-Spam-Detector/master/spam.csv -P ./parsetree-search

--2022-05-31 16:42:08--  https://raw.githubusercontent.com/susanli2016/SMS-Message-Spam-Detector/master/spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503673 (492K) [text/plain]
Saving to: ‘./parsetree-search/spam.csv.16’

spam.csv.16         100%[===================>] 491.87K  --.-KB/s    in 0.04s   

2022-05-31 16:42:09 (12.2 MB/s) - ‘./parsetree-search/spam.csv.16’ saved [503673/503673]



In [ ]:
%%file ./parsetree-search/app.py

from flask import Flask,render_template,flash, Response, redirect,url_for,session,logging,request,jsonify,make_response
import sqlite3
from flask_cors import CORS
import os
import threading
from pyngrok import ngrok
import numpy as np
import pandas
from pandas import DataFrame
import os.path
import re
import string
import datetime
import json
from flask import send_file
from io import BytesIO

import cx_Oracle      # We are an Oracle shop, and this changes some things
import csv
# import StringIO       # allows you to store response object in memory instead of on disk
from io import StringIO
# from flask import make_response # Necessary imports, should be obvious

import flaskcode

import nltk
from autocorrect import spell
from gensim.summarization import summarize as g_sumn

from sklearn.feature_extraction.text import CountVectorizer

os.environ["FLASK_ENV"] = "development"
app = Flask(__name__, static_folder='static')

CORS(app, resources=r'/api/*')

# app.debug = False
app.secret_key = 'key_app'

# app = Flask(__name__)
app.config.from_object(flaskcode.default_config)
# app.config['FLASKCODE_RESOURCE_BASEPATH'] = '/path/to/resource/folder'
app.config['FLASKCODE_RESOURCE_BASEPATH'] = './parsetree-search'
app.register_blueprint(flaskcode.blueprint, url_prefix='/vs')

def semua(args):
    import pandas as pd
    from nltk import Tree
    dataset = pd.read_table('Indonesian_Manually_Tagged_Corpus2.tsv', header=None, names=['words','tags'])
    #dataset.head(10)

    #Tulis kode untuk hitung frequency tiap jenis tag
    counted = dataset['tags'].value_counts()

    #Tulis kode untuk mengoutputkan 10 most frequent tags
    #print('10 most frequent tags:', counted.head(10), sep='\n')

    #Tulis kode untuk mengoutputkan 10 most frequent words given NN tag
    #NN_tagged = dataset[dataset['tags']=='NN']
    #NN_counted = NN_tagged['words'].value_counts()
    #print('\n10 most frequent words given NN tag:', NN_counted.head(10), sep='\n')

    # #Tulis kode untuk mengoutputkan 5 kata yang memiliki tag lebih dari 1, beserta tag2nya.
    # grouped_by_word = dataset.groupby('words')['tags'].unique()
    # word_and_mt1_tags = grouped_by_word[grouped_by_word.apply(lambda x: len(x)>1)]
    # print('\nWords and more than 1 tags with all tags:', word_and_mt1_tags.sample(5), sep='\n')

    import nltk
    from nltk import hmm
    def c_list_of_sentences(low, sep):
        sentence = []
        sentences = []
        for word in low:
            sentence.append(word)
            if word == sep:
                sentences.append(sentence)
                sentence = []
        return sentences

    #train_data adalah list yang menampung semua kalimat dengan format di atas
    #Tulis kode untuk transform kalimat ke format [('kata1','tag1'),('kata2','tag2'),('kata3','tag3')]
    #lalu simpan semua kalimat dalam list train_data

    list_of_words = dataset.to_records(index=False).tolist()
    sep = ('.', 'Z')

    train_data = c_list_of_sentences(list_of_words, sep)

    trainer=hmm.HiddenMarkovModelTrainer()
    tagger=trainer.train_supervised(train_data)

    grammar1 = nltk.data.load('file:mygrammar.cfg')
    parser = nltk.ChartParser(grammar1)

    def get_missing_words(grammar, tokens):
        """
        Find list of missing tokens not covered by grammar
        """
        missing = [tok for tok in tokens
                  if not grammar._lexical_index.get(tok)]
        return missing
    def parsetree (args):
        str_hasil = ""
        # cek_kata_diluar_grammar = get_missing_words(grammar1,args.split())
        cek_kata_diluar_grammar = ""
        if(len(cek_kata_diluar_grammar)==0):
          sent = args.split()
          # print(sent)

          tagged_text = tagger.tag(sent)
          #pos_tags = [pos for (token,pos) in tagged_text]
          # print (tagged_text)

          temp = ""

          #for tree in parser.parse(sent):
            #if tree != temp:
              #if str(tree.pretty_print()) is not None:
                #str_hasil+=str(tree.pretty_print())
            #temp = tree



          if(temp==''):
            sentence = tagged_text
            pattern = """NP: {<DT>?<JJ>*<NN>}
            VBD: {<VBD>}
            IN: {<IN>}"""
            NPChunker = nltk.RegexpParser(pattern)
            result = NPChunker.parse(sentence)
            if result is not None:
              str_hasil+=str(result)
            Tree.fromstring(str(result)).pretty_print()

            temp = result
          return temp

    return str(parsetree(args))

@app.route('/fphome')
def fphome():
    return render_template('fp_index2.html')

@app.route('/fppredict',methods=['POST'])
def fppredict():
    import numpy as np
    # import matplotlib.pyplot as plt
    import pandas as pd
    import os.path

    #BASE_DIR = os.path.dirname(os.path.abspath(__file__))
    #url = os.path.join(BASE_DIR, "spam.csv")

    BASE_DIR = os.getcwd()+'/parsetree-search'
    url = os.path.join(BASE_DIR, "spam.csv")


    df= pd.read_csv(url, encoding="latin-1")
    #df= pd.read_csv(url)
    #df = pd.read_csv(url)
    df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

    # Features and Labels
    df['label'] = df['class'].map({'ham': 0, 'spam': 1})
    X = df['message']
    y = df['label']

    # Extract Feature With CountVectorizer

    cv = CountVectorizer()

    X = cv.fit_transform(X) # Fit the Data

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    #Naive Bayes Classifier

    from sklearn.naive_bayes import MultinomialNB

    clf = MultinomialNB()
    clf.fit(X_train,y_train)
    clf.score(X_test,y_test)
    #Alternative Usage of Saved Model
    # joblib.dump(clf, 'NB_spam_model.pkl')
    # NB_spam_model = open('NB_spam_model.pkl','rb')

    # clf = joblib.load(NB_spam_model)

    if request.method == 'POST':
        message = request.form['message']
        data = [message]
        vect = cv.transform(data).toarray()
        my_prediction = clf.predict(vect)
        hasil_tree_sample = semua(message)

    def get_missing_words(grammar, tokens):
        """
        Find list of missing tokens not covered by grammar
        """
        missing = [tok for tok in tokens
                  if not grammar._lexical_index.get(tok)]
        return missing

    grammar1 = nltk.data.load('file:mygrammar.cfg')
    parser = nltk.ChartParser(grammar1)

    message = request.form['message']
    hasil_typo_sample = get_missing_words(grammar1, message.split())

    #return render_template('fp_result.html',prediction = my_prediction)
    return render_template('fp_index2.html',prediction = my_prediction, tree = hasil_tree_sample, typo = hasil_typo_sample)

@app.route("/")
def index():
    return redirect(url_for("login"))
    #return "Hello Flask IoT Simulator Using Python - Statistika Kelas B! :D"

@app.route("/chart")
def chart():
    BASE_DIR = os.getcwd()+'/parsetree-search'
    path_data = os.path.join(BASE_DIR, "data/data.csv")

    df = pandas.read_csv(path_data)
    days = np.unique(df['day'])

    allValues = {}
    for day in days:
        df_ = df.query('day==' + str(day))
        labels = df_['hour']
        #print(labels)
        values_dewp = df_['DEWP']
        values_temp = df_['TEMP']
        values1 = [values_dewp, values_temp, labels]
        allValues[day] = values1
    return render_template('charts.html', days = days, values=allValues)

@app.route("/ptree",methods=["GET", "POST"])
def ptree():
  tree_sample = """
                S
      __________|_________________________
    SUB         |        PEL             KET
     |          |         |               |
    FNOM       PRE      FPREP           FPREP
  ___|____      |     ____|_____      ____|______
 NN  CC   NN    VB   IN         NN   IN         NNP
 |   |    |     |    |          |    |           |
ibu dan  ayah pergi  ke       pasar  di       Jakarta

      """



  return tree_sample.replace(" ","&nbsp&nbsp").replace("\n","<br>")

@app.route("/login",methods=["GET", "POST"])
def login():
  conn = connect_db()
  db = conn.cursor()
  msg = ""
  if request.method == "POST":
      mail = request.form["mail"]
      passw = request.form["passw"]

      rs = db.execute("SELECT * FROM user WHERE Mail=\'"+ mail +"\'"+" AND Password=\'"+ passw+"\'" + " LIMIT 1")

      conn.commit()

      hasil = []
      for v_login in rs:
          hasil.append(v_login)

      if hasil:
          session['name'] = v_login[3]
          return redirect(url_for("fphome"))
      else:
          msg = "Masukkan Username (Email) dan Password dgn Benar!"

  return render_template("login.html", msg = msg)

@app.route("/register", methods=["GET", "POST"])
def register():
  conn = connect_db()
  db = conn.cursor()
  msg = ""
  if request.method == "POST":
      mail = request.form['mail']
      uname = request.form['uname']
      passw = request.form['passw']

      #cek apakah ada yg sudah menggunakan email tersebut
      rs = db.execute("SELECT * FROM user WHERE Mail=\'"+ mail +"\'" + " LIMIT 1")

      hasil = []
      for v_login in rs:
          hasil.append(v_login)

      if hasil:
          #msg = "Masukkan Username (Email) Lain, karena Email trsbt sudah digunakan oleh user Lainnya"
          msg = "Email trsbt sudah digunakan oleh user Lainnya"
          conn.commit()
          db.close()
          conn.close()
      else:
          if (bool(mail and not mail.isspace()) and bool(uname and not uname.isspace()) and bool(passw and not passw.isspace())):
              cmd = "insert into user(Mail, Password,Name,Level) values('{}','{}','{}','{}')".format(mail,passw,uname,'1')
              conn.execute(cmd)
              conn.commit()
              db.close()
              conn.close()
              #return redirect(url_for("login"))
              msg = "Masukkan Username (Email) dan Password dgn Benar!"
              return render_template("login.html", msg = msg)
          else:
              msg = "Lengkapi isian data Anda!"
  return render_template("register.html", msg = msg)

def connect_db():
    # import os.path
    # BASE_DIR = os.path.dirname(os.path.abspath(__file__))
    BASE_DIR = os.getcwd()+'/parsetree-search'
    db_path = os.path.join(BASE_DIR, "data.db")
    return sqlite3.connect(db_path)

@app.errorhandler(404)
def page_not_found(error):
    return render_template("404.html")

@app.errorhandler(500)
def internal_server_error(error):
    return render_template("500.html")

@app.route('/getsuhu_tipe_satu', methods=["GET", "POST"])
def getsuhu_tipe_satu():

    if 'name' in session:
        name = session['name']
    else:
        name = 'Guest'

    from datetime import datetime
    import pytz
    Date = str(datetime.today().astimezone(pytz.timezone('Asia/Jakarta')).strftime('%d-%m-%Y %H:%M:%S'))

    conn = connect_db()
    db = conn.cursor()

    c = db.execute(""" SELECT * FROM  data_suhu_dll_api_openweathermap """)

    mydata = c.fetchall()
    for x in c.fetchall():
        name_v=x[0]
        data_v=x[1]
        break

    hasil = []
    for v_login in c:
        hasil.append(v_login)

    conn.commit()
    db.close()
    conn.close()

    return render_template("getsuhu_tipe_satu.html", header = mydata)

@app.route('/unduh_data_tipe_satu/', methods=["GET"])
def dw_data_tipe_satu():
    # name = request.args.get('name')
    conn = connect_db()
    db = conn.cursor()

    c = db.execute("SELECT * FROM data_suhu_dll_api_openweathermap")

    # def export(load_file_id):
    si = StringIO()
    cw = csv.writer(si)

    rows = c.fetchall()
    cw.writerow([i[0] for i in c.description])
    cw.writerows(rows)
    response = make_response(si.getvalue())
    response.headers['Content-Disposition'] = 'attachment; filename=data.csv'
    response.headers["Content-type"] = "text/csv"

    conn.commit()
    db.close()
    conn.close()

    return response

@app.route('/logout')
def logout():
   # remove the name from the session if it is there
   session.pop('name', None)
   return redirect(url_for('index'))

@app.route("/p2021",methods=["GET", "POST"])
def p2021():
  conn = connect_db()
  db = conn.cursor()
  msg = ""

  return render_template("docs/assets/index.html", msg = msg)

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

Overwriting ./parsetree-search/app.py


In [ ]:
# cara run Web App
!echo yes | freeport 5000
output.clear()
get_ipython().system_raw('python ./parsetree-search/app.py --host=0.0.0.0 --port=5000 &')

In [ ]:
from pyngrok import ngrok

#terminate open tunnels if exist
ngrok.kill()

#Setting the authtoken (optional)
#Get your authtoken from htpps://dashboard.ngrok.com/auth
# NGROK_AUTH_TOKEN_ALT = ""
# !ngrok tcp 5000
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.connect(5000)

<NgrokTunnel: "http://ab32-35-185-31-224.ngrok.io" -> "http://localhost:5000">

Contoh ilustrasi Hasil ParseTree dan Get Typo:
![]( https://docs.google.com/uc?export=download&id=1X7qldS1-JckrkpOVqZQLH0DzqOUczHPS)

# Start Buat Koding FP (Dgn C/C++ || Java || Python)

In [ ]:
#
# Koding ParseTree disini prototype-nya sebelum dipindahkan ke Web App |
# (ambil koding dari solusi soal UTS no.2 + ambil dari koding dasar dari link: https://bit.ly/3w548Eb lalu modifikasi untuk get "isi abstrak dok")
# yg di ParseTree-kan cukup dari kalimat-kalimat yang ada pada abstrak indo saja atau abtract yg english saja

# ..




### untuk contoh koding ParseTree

In [ ]:
!gdown 1n4DjHpxXWFbKeH8D2oK7sYY5RDy9n_6b
!gdown 1V-vhHTJ5kaXEOIPEjxwa39nOXslsVMdU

Downloading...
From: https://drive.google.com/uc?id=1n4DjHpxXWFbKeH8D2oK7sYY5RDy9n_6b
To: /content/drive/MyDrive/#Pengajaran Genap 2021-2022/ASD For Me/MyApp/MyFP/Indonesian_Manually_Tagged_Corpus2.tsv
100% 2.48M/2.48M [00:00<00:00, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=1V-vhHTJ5kaXEOIPEjxwa39nOXslsVMdU
To: /content/drive/MyDrive/#Pengajaran Genap 2021-2022/ASD For Me/MyApp/MyFP/mygrammar.cfg
100% 238k/238k [00:00<00:00, 54.6MB/s]


In [ ]:
import pandas as pd
from nltk import Tree
dataset = pd.read_table('Indonesian_Manually_Tagged_Corpus2.tsv', header=None, names=['words','tags'])
#dataset.head(10)

#Tulis kode untuk hitung frequency tiap jenis tag
counted = dataset['tags'].value_counts()

#Tulis kode untuk mengoutputkan 10 most frequent tags
#print('10 most frequent tags:', counted.head(10), sep='\n')

#Tulis kode untuk mengoutputkan 10 most frequent words given NN tag
#NN_tagged = dataset[dataset['tags']=='NN']
#NN_counted = NN_tagged['words'].value_counts()
#print('\n10 most frequent words given NN tag:', NN_counted.head(10), sep='\n')

# #Tulis kode untuk mengoutputkan 5 kata yang memiliki tag lebih dari 1, beserta tag2nya.
# grouped_by_word = dataset.groupby('words')['tags'].unique()
# word_and_mt1_tags = grouped_by_word[grouped_by_word.apply(lambda x: len(x)>1)]
# print('\nWords and more than 1 tags with all tags:', word_and_mt1_tags.sample(5), sep='\n')

import nltk
from nltk import hmm

# Fungsi membuat list of list of tuple
# low: List of Words
# sep: separator (pemisah dalam kalimat, titik)
def c_list_of_sentences(low, sep):
    sentence = []
    sentences = []
    for word in low:
        sentence.append(word)
        if word == sep:
            sentences.append(sentence)
            sentence = []
    return sentences

#train_data adalah list yang menampung semua kalimat dengan format di atas
#Tulis kode untuk transform kalimat ke format [('kata1','tag1'),('kata2','tag2'),('kata3','tag3')]
#lalu simpan semua kalimat dalam list train_data

list_of_words = dataset.to_records(index=False).tolist()
sep = ('.', 'Z')

train_data = c_list_of_sentences(list_of_words, sep)

trainer=hmm.HiddenMarkovModelTrainer()
tagger=trainer.train_supervised(train_data)

grammar1 = nltk.data.load('file:mygrammar.cfg')
parser = nltk.ChartParser(grammar1)

def get_missing_words(grammar, tokens):
    """
    Find list of missing tokens not covered by grammar
    """
    missing = [tok for tok in tokens
               if not grammar._lexical_index.get(tok)]
    return missing


def parsetree (args):
    str_hasil = ""
    cek_kata_diluar_grammar = get_missing_words(grammar1,args.split())
    if(len(cek_kata_diluar_grammar)==0):
      sent = args.split()
      # print(sent)

      tagged_text = tagger.tag(sent)
      pos_tags = [pos for (token,pos) in tagged_text]
      # print (tagged_text)

      temp = ""

      for tree in parser.parse(sent):
        if tree != temp:
          if str(tree.pretty_print()) is not None:
            str_hasil+=str(tree.pretty_print())
        temp = tree



      if(temp==''):
        sentence = tagged_text
        pattern = """NP: {<DT>?<JJ>*<NN>}
        VBD: {<VBD>}
        IN: {<IN>}"""
        NPChunker = nltk.RegexpParser(pattern)
        result = NPChunker.parse(sentence)
        if result is not None:
          str_hasil+=str(result)
        Tree.fromstring(str(result)).pretty_print()

        temp = result



      return temp
    else:
      str_hasil += "Terdapat kata yang tidak ada pada grammer = " + str(cek_kata_diluar_grammar) + "\n"
      # print("Terdapat kata yang tidak ada pada grammer = ", cek_kata_diluar_grammar)
      for term_in in cek_kata_diluar_grammar:
        args = args.replace(term_in,"")

      sent = args.split()
      # print(sent)

      tagged_text = tagger.tag(sent)
      pos_tags = [pos for (token,pos) in tagged_text]
      # print (tagged_text)

      temp = ""

      for tree in parser.parse(sent):
        if tree != temp:
          if str(tree.pretty_print()) is not None:
            str_hasil += str(tree.pretty_print())
        temp = tree

      if(temp==''):
        sentence = tagged_text
        pattern = """NP: {<DT>?<JJ>*<NN>}
        VBD: {<VBD>}
        IN: {<IN>}"""
        NPChunker = nltk.RegexpParser(pattern)
        result = NPChunker.parse(sentence)
        if result is not None:
          str_hasil += str(result)
        Tree.fromstring(str(result)).pretty_print()

      return str_hasil




#parsetree("saya membaca buku tiga halaman di halaman sekolah")
#parsetree("kami bangkit bersama dengan keterpurukan")
# temp = parsetree("ibu dan ayah pergi ke pasar di Jakarta")

toText = parsetree("ibu dan ayah pergi ke pasar di Jakarta")
# toText = parsetree("banjir sudah melanda kota Jakarta tiga tahun silam")
# parsetree("kami bangkit bersama dengan keterpurukan")

                S                                    
      __________|_________________________            
    SUB         |        PEL             KET         
     |          |         |               |           
    FNOM       PRE      FPREP           FPREP        
  ___|____      |     ____|_____      ____|______     
 NN  CC   NN    VB   IN        NNP   IN         NNP  
 |   |    |     |    |          |    |           |    
ibu dan  ayah pergi  ke       pasar  di       Jakarta

                S                                    
      __________|_________________________            
    SUB         |        PEL             KET         
     |          |         |               |           
    FNOM       PRE      FPREP           FPREP        
  ___|____      |     ____|_____      ____|______     
 NN  CC   NN    VB   IN        NNP   IN         NNP  
 |   |    |     |    |          |    |           |    
ibu dan  ayah pergi  ke       pasar  di       Jakarta

                S 

In [ ]:
### CREATE VIRTUAL DISPLAY ###
!pip install ghostscript
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0.

In [ ]:
parsetree("Buku Pedoman Akademik FILKOM Universitas Brawijaya merupakan suatu kebutuhan informasi akademik yang cukup penting, dan juga buku penunjang pembelajaran seperti Free e-Book bagi para mahasiswa")

Terdapat kata yang tidak ada pada grammer =  ['Akademik', 'FILKOM', 'Brawijaya', 'akademik', 'penting,', 'Free', 'e-Book']
['Buku', 'Pedoman', 'Universitas', 'merupakan', 'suatu', 'kebutuhan', 'informasi', 'yang', 'cukup', 'dan', 'juga', 'buku', 'penunjang', 'pembelajaran', 'seperti', 'bagi', 'para', 'mahasiswa']
[('Buku', 'NN'), ('Pedoman', 'NNP'), ('Universitas', 'NNP'), ('merupakan', 'VB'), ('suatu', 'CD'), ('kebutuhan', 'NN'), ('informasi', 'NN'), ('yang', 'SC'), ('cukup', 'RB'), ('dan', 'CC'), ('juga', 'RB'), ('buku', 'NN'), ('penunjang', 'NN'), ('pembelajaran', 'NN'), ('seperti', 'IN'), ('bagi', 'IN'), ('para', 'DT'), ('mahasiswa', 'NN')]
                                                                                     S                                                                                                              
      _______________________________________________________________________________|________________________________________________________________

''

In [ ]:
str(toText)

'(S\n  (SUB (FNOM (NN ibu) (CC dan) (NN ayah)))\n  (PRE (VB pergi))\n  (PEL (FPREP (IN ke) (NN pasar)))\n  (KET (FPREP (IN di) (NNP Jakarta))))'

In [ ]:
from nltk import Tree
Tree.fromstring(str(toText)).pretty_print()

                S                                    
      __________|_________________________            
    SUB         |        PEL             KET         
     |          |         |               |           
    FNOM       PRE      FPREP           FPREP        
  ___|____      |     ____|_____      ____|______     
 NN  CC   NN    VB   IN         NN   IN         NNP  
 |   |    |     |    |          |    |           |    
ibu dan  ayah pergi  ke       pasar  di       Jakarta



In [ ]:
!pip install treelib
output.clear()

In [ ]:
from treelib import Node, Tree


lst = ["S", ["NP", ["DT", "the"], ["NN", "dog"]], ["VP", ["Vt", "saw"], ["NP", ["NP", ["DT", "the"], ["NN", "man"]],
                                                                   ["PP", ["P", "with"],
                                                                    ["NP", ["DT", "the"], ["NN", "telescope"]]]]]]

# lst = [('ibu', 'NN'), ('dan', 'CC'), ('ayah', 'NN'), ('pergi', 'VB'), ('ke', 'IN'), ('pasar', 'NN'), ('di', 'IN'), ('Jakarta', 'NNP')]

root, *tail = lst
tree = Tree()
node = Node(root)
tree.add_node(node)

q = [[node, *tail]]
while q:
    parent, *children = q.pop()
    for child in children:
        if isinstance(child, list):
            head, *tail = child
            node = tree.create_node(head, parent=parent)
            q.append([node, *tail])
        else:
            tree.create_node(child, parent=parent)

tree.show()

S
├── NP
│   ├── DT
│   │   └── the
│   └── NN
│       └── dog
└── VP
    ├── NP
    │   ├── NP
    │   │   ├── DT
    │   │   │   └── the
    │   │   └── NN
    │   │       └── man
    │   └── PP
    │       ├── NP
    │       │   ├── DT
    │       │   │   └── the
    │       │   └── NN
    │       │       └── telescope
    │       └── P
    │           └── with
    └── Vt
        └── saw



In [ ]:
#
# Koding "Typo atau get kata yg tidak ada pada kamus" disini prototype-nya sebelum dipindahkan ke Web App |
# (ambil dari koding dasar dari link: https://bit.ly/3w548Eb lalu modifikasi utk get "Typo atau get kata yg tidak ada pada kamus")

# ..


# Hi All Great Students

Semoga Sukses. Aamiin. :D
![]( https://docs.google.com/uc?export=download&id=1vJNmuncRehLc3WCZkFYCU6rRMVTy-w0k)